# import libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
import traceback
import re

# Load files with news urls

In [4]:
urls_df =  pd.read_excel("news_id_url.xlsx")

# Extract accounts in news urls

In [5]:
# this step is previuesly in a spearte python code . the file name are "scraping_beamreport_accounts_final.py"

In [6]:
accounts = []
unlabeled_news = []

news_num = urls_df.shape[0]
batch_size = 50
loops_num = math.ceil(news_num / batch_size)

df_counter = 0

for i in range(loops_num):
    counter = 0
    error_log_filename = f"accounts_data/error_log_file_{i + 1}.txt"
    
    with open(error_log_filename, "w", encoding="utf-8") as log_file:
        while counter < batch_size and df_counter < news_num:
            try:
                current_news_id = urls_df.at[df_counter,"news_id"]
                current_url = urls_df.at[df_counter, 'link']
                
                response = requests.get(current_url, timeout=10)
                response.raise_for_status()
                html_text = response.text
                soup = BeautifulSoup(html_text, 'lxml')

                temp_divs = soup.find_all(
                    'div',
                    attrs={
                        'data-element_type': "widget",
                        'data-widget_type': "icon-list.default"
                    }
                )
                
                try:
                    label = temp_divs[1].text.strip()
                    
                    if not label:
                        raise ValueError("Label text is empty.")
                except:
                    unlabeled_news.append(current_news_id)
                    df_counter += 1
                    counter += 1
                    continue
                        
                table = soup.find('table')
                if table is None:
                    log_file.write(f"[MissingTable] News ID: {current_news_id} | URL: {current_url}\n")
                    df_counter += 1
                    counter += 1
                    continue
                
                tr_elements = table.find_all('tr')
                
                if len(tr_elements) == 0:
                    log_file.write(f"[EmptyTable] News ID: {current_news_id} | URL: {current_url}\n")
                    df_counter += 1
                    counter += 1
                    continue
                
                for r,tr in enumerate(tr_elements):
                    try:
                        td_elements = tr.find_all('td')
                            
                        account_info = {
                            'row_table_number': r+1,
                            'account_url': 'undefined',
                            'account_name': 'undefined',
                            'followers_number': 'undefined',
                            'label': temp_divs[1].text.strip(),
                            'news_id': current_news_id
                        }

                        def clean_text(element):
                            """Safely extract text content from an HTML element, replacing &nbsp; and stripping whitespace."""
                            if element:
                                text = element.get_text()
                                if text and text.replace('\xa0', '').strip():  # Check if not just &nbsp;
                                    return text.replace('\xa0', ' ').strip()
                            return 'undefined'


                        account_info['account_url'] = td_elements[1].select_one('a')['href'] if td_elements[1].select_one('a') else 'undefined'
                        account_info['account_name'] = clean_text(td_elements[1])
                        account_info['followers_number'] = clean_text(td_elements[2])

                        accounts.append(account_info)
                
                    except Exception as e:
                        log_file.write(f"[RowParseError] News ID: {current_news_id} | {current_url} | {str(e)}\n")
                        continue

            except requests.exceptions.RequestException as net_err:
                # Network error: save current accounts and exit
                log_file.write(f"[NetworkError] News ID: {current_news_id} | URL: {current_url} | {str(net_err)}\n")
                log_file.write("Network error occurred. Saving current progress and exiting.\n")

                df = pd.DataFrame(accounts)
                df.to_csv("accounts_info_uncompleted.csv", index=False, encoding='utf-8-sig')
                print("Network error. Progress saved to 'accounts_info_uncompleted.csv'. Exiting.")
                exit()

            except Exception as e:
                # Any other unexpected error
                log_file.write(f"[UnknownError] News ID: {current_news_id} | URL: {current_url}\n")
                log_file.write(traceback.format_exc() + "\n")

            df_counter += 1
            counter += 1

    # Save the accounts of this batch
    print(f"\nBatch {i + 1} done. Accounts collected so far: {len(accounts)}")
    df = pd.DataFrame(accounts)
    df.to_csv(f"accounts_data/accounts_info_file_{i+1}.csv", index=False, encoding='utf-8-sig')
    accounts = []


Batch 1 done. Accounts collected so far: 148

Batch 2 done. Accounts collected so far: 165

Batch 3 done. Accounts collected so far: 132

Batch 4 done. Accounts collected so far: 138

Batch 5 done. Accounts collected so far: 177

Batch 6 done. Accounts collected so far: 266

Batch 7 done. Accounts collected so far: 26

Batch 8 done. Accounts collected so far: 126

Batch 9 done. Accounts collected so far: 174

Batch 10 done. Accounts collected so far: 111

Batch 11 done. Accounts collected so far: 77


# Merge account files in one file 

In [7]:
num_accounts_files = 11
dfs_holder =[]

for i in range(1,num_accounts_files+1):
    df = pd.read_csv(f"accounts_data/accounts_info_file_{i}.csv")
    dfs_holder.append(df)
    
main_df = pd.concat(dfs_holder,ignore_index=True)
#main_df['news_id'] = pd.to_numeric(main_df['news_id'], errors='coerce')
main_df = main_df.sort_values(by=["news_id","row_table_number"],ascending=[True,True])

main_df.to_csv("accounts_data/accounts_info_file_merged.csv", index=False, encoding='utf-8-sig')


# saving a file containing news with no labels

In [8]:
no_labels_df = pd.DataFrame(unlabeled_news)

print("number of news with no labels exist: ",no_labels_df.shape[0])
no_labels_df.to_csv("accounts_data/news_no_labels_file.csv", index=False, encoding='utf-8-sig')

number of news with no labels exist:  65


# analysis values of columns

In [10]:
#loading last account file saved so far

main_df = pd.read_csv("accounts_data/accounts_info_file_merged.csv")
print("number of rows in main_df: ",main_df.shape[0])
main_df.head()

number of rows in main_df:  1540


,row_table_number,account_url,account_name,followers_number,label,news_id
0,1,https://www.facebook.com/share/p/1AytsiHgrd/,قوات العمل الخاص هيئة العمليات,(697) ألف متابع,مضلل,1
1,2,https://www.facebook.com/share/p/15pwXprDjz/,عروض تذاكر بدر وتاركو للطيران,(319) ألف متابع,مضلل,1
2,3,https://www.facebook.com/share/p/1DvNK8fbi9/,ارصاد منذر أحمد الحاج,(89) ألف متابع,مضلل,1
3,1,https://www.facebook.com/share/p/1EjSjxii6k/,قوات العمل الخاص هيئة العمليات,1,مفبرك,2
4,2,https://www.facebook.com/share/p/1GGMLwQhjv/,شبكة لمتنا الإخبارية,2,مفبرك,2


Now, we will look at undefined values in the file and in which columns it appears to know more about how to overcome extraction problems (if possible). four columns can be analyzed, 

In [11]:
def evaluate_df(df):
    
    #label column
    condition = (df['label'].isna()) | (df['label'].astype(str).str.strip().str.len() == 0)

    print("number of undefined labels: ",condition.sum())
    #account_name column
    condition = df['account_name'] == 'undefined'
    print("number of undefined accounts name : ",condition.sum())

    #followers_number column
    condition = df['followers_number'] == 'undefined'
    print("number of undefined followers_number : ",condition.sum())

    #account_url column
    condition = df['account_url'] == 'undefined'
    print("number of undefined account_url : ",condition.sum())
    

evaluate_df(main_df)

number of undefined labels:  0
number of undefined accounts name :  3
number of undefined followers_number :  4
number of undefined account_url :  86


### remove unwanted rows

There are unwanted rows in the main_df, it can be identified using certain values in followers number column

In [12]:
unwanted_vals = ['عدد المتابعات','عدد المتابعين \\ الأعضاء','عدد المتابعيين','عدد المتابعين']

condition = main_df["followers_number"].isin(unwanted_vals)
print("number of unwanted rows in main_df: ",condition.sum())

#see a sample from this rows
main_df[condition].head()

number of unwanted rows in main_df:  77


,row_table_number,account_url,account_name,followers_number,label,news_id
398,1,undefined,الصفحة/ الموقع الإلكتروني,عدد المتابعين,مضلل,132
716,1,undefined,اسم الصفحة \ الحساب,عدد المتابعين,مضلل,240
739,1,undefined,الحساب \ الصفحة\ المجموعة,عدد المتابعين \ الأعضاء,مفبرك,246
763,1,undefined,اسم الحساب \ الصفحة,عدد المتابعين,مفبرك,252
767,1,undefined,اسم الصفحة \ الحساب,عدد المتابعين,انتقائي,253


#### remove them from main_df

In [13]:
#remove them from main_df
main_df = main_df[~condition].reset_index(drop=True)
print("number of rows in main_df after removing unwanted rows: ",main_df.shape[0])

number of rows in main_df after removing unwanted rows:  1463


In [14]:
evaluate_df(main_df)

number of undefined labels:  0
number of undefined accounts name :  3
number of undefined followers_number :  4
number of undefined account_url :  9


### look at news urls with undefined account_url

In [15]:
condition = (main_df['account_url'] == 'undefined')

main_df[condition].head(9)

,row_table_number,account_url,account_name,followers_number,label,news_id
790,7,undefined,undefined,undefined,مضلل,258
808,6,undefined,undefined,undefined,مفبرك,262
873,8,undefined,undefined,undefined,مضلل,274
1385,1,undefined,CH-3,CH-4,مفبرك,499
1386,2,undefined,256 كلم\الساعة,180 كلم\الساعة,مفبرك,499
1387,3,undefined,2400 كلم,5000 كلم,مفبرك,499
1388,4,undefined,180 كجم,350 كجم,مفبرك,499
1389,5,undefined,12 ساعة,30-40 ساعة,مفبرك,499
1390,6,undefined,صاروخ جو أرض (AR-1),صواريخ جو أرض موجهة بالليزرقنابل موجهة بالليزر...,مفبرك,499


In [16]:
# look at news urls with undefined account_url

condition = (main_df['account_url'] == 'undefined')
news_id_list = list(main_df[condition]['news_id'].unique())

undefined_urls_df = urls_df[urls_df["news_id"].isin(news_id_list)]
news_num = undefined_urls_df.shape[0]

for i in range(min(10,news_num)):
    print(undefined_urls_df['news_id'].iloc[i])
    print(undefined_urls_df['link'].iloc[i])

258
https://www.beamreports.com/2024/03/09/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%b5%d9%88%d8%b1-%d8%ae%d8%b1%d9%88%d8%ac-%d8%a7%d9%84%d8%a8%d8%b4%d9%8a%d8%b1-%d9%85%d9%86-%d9%85%d8%b3%d8%aa%d8%b4%d9%81%d9%89-%d8%b9%d9%84/
262
https://www.beamreports.com/2024/02/26/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%ae%d8%b7%d8%a7%d8%a8-%d8%aa%d8%b9%d9%8a%d9%8a%d9%86-%d9%82%d9%8a%d8%a7%d8%af%d8%a7%d8%aa-%d9%85%d9%86-%d8%ad%d8%b1%d9%83%d8%a9-%d8%a7%d9%84%d8%b9/
274
https://www.beamreports.com/2024/01/28/%d9%85%d8%a7-%d8%b5%d8%ad%d8%a9-%d9%82%d8%b1%d8%a7%d8%b1-%d9%84%d9%84%d9%85%d8%ad%d9%83%d9%85%d8%a9-%d8%a7%d9%84%d8%b9%d9%84%d9%8a%d8%a7-%d9%8a%d9%82%d8%b6%d9%8a-%d8%a8%d8%b9%d9%88%d8%af/
499
https://www.beamreports.com/2022/03/16/%d9%85%d8%a7-%d8%b5%d8%ad%d8%a9-%d8%a7%d9%84%d9%85%d9%86%d8%b4%d9%88%d8%b1%d8%a7%d8%aa-%d9%88%d8%a7%d9%84%d8%b5%d9%88%d8%b1-%d8%a7%d9%84%d8%b1%d8%a7%d8%a6%d8%ac%d8%a9-%d8%ad%d9%88%d9%84-%d8%b5%d9%86/


be look at these four articles we found that :
- news with id 499 are labeled news containing table that is not for accounts. so 6 rows must be removed
- news with id 274, 262 and 258 has empty rows in a table. so 3 rows must be removed

we will:
- remove empty rows
- put new with id 499 in a file called labeled_news_with_no_tables.csv

In [17]:
temp_label = main_df[main_df["news_id"] == 499]["label"].iloc[0]
temp_news_info= [{'news_id': 499,'label':temp_label}]
labeled_news_no_tables_df = pd.DataFrame(temp_news_info)

labeled_news_no_tables_df.to_csv("accounts_data/labeled_news_with_no_tables.csv", index=False, encoding='utf-8-sig')

In [18]:
# removing rows with undefined urls
condition = main_df["account_url"] == 'undefined'
main_df = main_df[~condition].reset_index(drop=True)

print("number of rows in main_df after removing unwanted rows: ",main_df.shape[0])

number of rows in main_df after removing unwanted rows:  1454


In [19]:
evaluate_df(main_df)

number of undefined labels:  0
number of undefined accounts name :  0
number of undefined followers_number :  1
number of undefined account_url :  0


In [20]:
# see what row with undefined follower_number

condition = main_df["followers_number"] == 'undefined'
main_df[condition].head()

print(urls_df[urls_df["news_id"] == 381]["link"].iloc[0])

https://www.beamreports.com/2023/04/15/%d9%85%d8%a7%d8%b5%d8%ad%d8%a9-%d8%a7%d9%84%d8%a8%d9%8a%d8%a7%d9%86-%d8%a7%d9%84%d9%85%d9%86%d8%b3%d9%88%d8%a8-%d8%a5%d9%84%d9%89-%d9%86%d8%a7%d8%b8%d8%b1-%d8%a7%d9%84%d8%b1%d8%b2%d9%8a%d9%82%d8%a7/


we will not remove this row

In [21]:
# save main_df
main_df.to_csv("accounts_data/accounts_info_file_filtered_1.csv", index=False, encoding='utf-8-sig')

# Analyzing error log files

In [89]:
import re


def extract_info_from_logs(path,pattern,start_idx,n):
    '''
    extracting news_id and error type from error log files
    '''
    
    error_types = []
    news_ids = []

    for i in range(start_idx, n+1):
        filename = pattern + str(i)+".txt"
        try:
            with open(path+filename, 'r', encoding='utf-8') as file:
                content = file.read()

                # Match pattern like: [ErrorType] News ID: 123
                matches = re.findall(r'\[([^\[\]]+)\]\s+News ID\s*:\s*(\d+)', content)

                for error_type, news_id in matches:
                    error_types.append(error_type)
                    news_ids.append(int(news_id))

        except FileNotFoundError:
            print(f"File not found: {filename}")
        except Exception as e:
            print(f"Error reading {filename}: {e}")

    return error_types, news_ids


extract news_id and error types from error log files

In [91]:
import os

directory = os.getcwd()+"/accounts_data/"
err_file_patt_string = "error_log_file_"
start_idx = 1
num_log_error_files = 11
err_type_list, err_news_id_list = extract_info_from_logs(directory,err_file_patt_string,start_idx,num_log_error_files)

err_types = list(set(err_type_list))
err_news_id = list(set(err_news_id_list))

print("unique error types: ", err_types)
print("number of unique news in the log error file: ",len(err_news_id))

unique error types:  ['MissingTable', 'RowParseError']
number of unique news in the log error file:  166


In [92]:
news_err_row = [news_id for news_id,err_type in zip(err_news_id_list,err_type_list) if err_type == 'RowParseError']
news_err_row = list(set(news_err_row))
news_err_table= list(set(err_news_id)-set(news_err_row))

print("news with row error:",len(news_err_row))
print("news with table error: ",len(news_err_table))

news with row error: 13
news with table error:  153


### investigating news with Table errors:

In [93]:
condition = urls_df["news_id"].isin(news_err_table)
df_mis_table_urls = urls_df[condition].reset_index(drop=True).sort_values(by="news_id",ascending=True)


for i in range(4):
    print(df_mis_table_urls["link"].iloc[i])

https://www.beamreports.com/2025/06/27/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%b5%d9%88%d8%b1%d8%a9-%d8%a7%d9%84%d8%a8%d8%a7%d8%b5%d8%a7%d8%aa-%d8%a7%d9%84%d8%b3%d9%81%d8%b1%d9%8a%d8%a9-%d8%a7%d9%84%d8%b9%d8%a7%d8%a6%d8%af-2/
https://www.beamreports.com/2025/01/24/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%a7%d9%84%d8%aa%d8%b5%d8%b1%d9%8a%d8%ad-%d8%a7%d9%84%d9%85%d8%aa%d8%af%d8%a7%d9%88%d9%84-%d8%b9%d9%86-%d8%ad%d8%a7%d9%83%d9%85-%d8%a5%d9%82%d9%84%d9%8a/
https://www.beamreports.com/2024/08/31/%d9%85%d8%a7-%d8%b5%d8%ad%d8%a9-%d8%a7%d8%af%d8%b9%d8%a7%d8%a1-%d8%a7%d9%84%d8%af%d8%b9%d9%85-%d8%a7%d9%84%d8%b3%d8%b1%d9%8a%d8%b9-%d8%a7%d9%84%d8%a5%d9%81%d8%b1%d8%a7%d8%ac-%d8%b9%d9%86/
https://www.beamreports.com/2024/08/14/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%a7%d9%84%d8%b5%d9%88%d8%b1-%d8%a7%d9%84%d9%85%d8%aa%d8%af%d8%a7%d9%88%d9%84%d8%a9-%d9%84%d9%84%d9%82%d8%a8%d8%b6-%d8%b9%d9%84%d9%89-%d9%85%d8%b5%d8%b1/


After reviewing some sample news items with table errors, we found that they are either reports and investigations that do not contain a news table, or actual news articles that are simply missing the table.

We will separate the news articles with missing tables into a separate file for further investigation. These can be identified by the presence of labels.

In [94]:
labeled_news_info_list = []
unlabeled_news_info_list=[]

news_link_pairs = zip(df_mis_table_urls["news_id"],df_mis_table_urls["link"])

for crr_news_id,current_url in news_link_pairs: 

    response = requests.get(current_url, timeout=10)
    response.raise_for_status()
    html_text = response.text
    soup = BeautifulSoup(html_text, 'lxml')
    
    temp_divs = soup.find_all(
        'div',
        attrs={
            'data-element_type': "widget",
            'data-widget_type': "icon-list.default"
        }
    )

    try:
        label = temp_divs[1].text.strip()

        if label:
            news_info ={
                'label': label,
                'news_id': crr_news_id
            }
            labeled_news_info_list.append(news_info)
        else:
            raise ValueError("Label text is empty.")
    except:
        unlabeled_news_info_list.append(current_news_id)
        continue

print("number of extracted labeled_news with no tables: ",len(labeled_news_info_list))
print("number of extracted ulabeled_news with no tables: ",len(unlabeled_news_info_list))

number of extracted labeled_news with no tables:  153
number of extracted ulabeled_news with no tables:  0


- saving news with no labels in a file called "news_no_labels_file.csv"
- saving news with lables and without table in file called "labeled_news_with_no_tables.csv"

In [95]:
temp_df = pd.DataFrame(labeled_news_info_list)

temp_df.head()

,label,news_id
0,مفبرك,19
1,مفبرك,100
2,مضلل,159
3,مضلل,170
4,مضلل,178


In [96]:
labeled_news_no_tables_df = pd.read_csv("accounts_data/labeled_news_with_no_tables.csv")

labeled_news_no_tables_df = pd.concat([labeled_news_no_tables_df,temp_df]).reset_index(drop=True).sort_values(by="news_id",ascending=True)
labeled_news_no_tables_df.to_csv("accounts_data/labeled_news_with_no_tables.csv", index=False, encoding='utf-8-sig')

### investigating news with row errors:

looking at the structure of rows to identify new way to capture information from a table

In [97]:
condition = urls_df["news_id"].isin(news_err_row)
df_row_err_urls = urls_df[condition].reset_index(drop=True).sort_values(by="news_id",ascending=True)

for i in range(len(news_err_row)):
    print(df_row_err_urls["news_id"].iloc[i])
    print(df_row_err_urls["link"].iloc[i])

22
https://www.beamreports.com/2025/06/19/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%aa%d8%b5%d8%b1%d9%8a%d8%ad-%d9%85%d9%86%d8%b3%d9%88%d8%a8-%d9%84%d8%b1%d8%a6%d9%8a%d8%b3-%d8%a7%d9%84%d9%88%d8%b2%d8%b1%d8%a7%d8%a1-%d9%83%d8%a7%d9%85/
63
https://www.beamreports.com/2025/04/04/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%aa%d9%88%d8%b9%d9%91%d8%af-%d9%88%d8%b2%d9%8a%d8%b1-%d8%a7%d9%84%d8%af%d9%81%d8%a7%d8%b9-%d8%a7%d9%84%d9%85%d8%b5%d8%b1%d9%8a-%d8%a8%d8%a7%d9%84%d8%aa/
95
https://www.beamreports.com/2025/02/02/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d8%a7%d9%84%d9%81%d9%8a%d8%af%d9%8a%d9%88-%d8%a7%d9%84%d9%85%d8%aa%d8%af%d8%a7%d9%88%d9%84-%d8%b9%d9%84%d9%89-%d8%a3%d9%86%d9%87-%d8%a7%d8%b4%d8%aa%d8%a8/
144
https://www.beamreports.com/2024/09/28/%d9%85%d8%a7-%d8%ad%d9%82%d9%8a%d9%82%d8%a9-%d9%85%d9%82%d8%b7%d8%b9-%d8%a7%d9%84%d9%81%d9%8a%d8%af%d9%8a%d9%88-%d8%a7%d9%84%d9%85%d8%aa%d8%af%d8%a7%d9%88%d9%84-%d8%b9%d9%84%d9%89-%d8%a3%d9%86%d9%87-6/
250
https://www.beamreports.c

after looking at some pages,there is tables that used the tag < th > instead of < td > , and there is a tables with two columns (account and number of followers)instead of three

In [99]:
accounts = []
unlabeled_news = []
labeled_news_no_accounts = []

news_link_pairs = zip(df_row_err_urls["news_id"], df_row_err_urls["link"])

for current_news_id, current_url in news_link_pairs: 

    response = requests.get(current_url, timeout=10)
    response.raise_for_status()
    html_text = response.text
    soup = BeautifulSoup(html_text, 'lxml')

    temp_divs = soup.find_all(
        'div',
        attrs={
            'data-element_type': "widget",
            'data-widget_type': "icon-list.default"
        }
    )

    try:
        label = temp_divs[1].get_text(strip=True) if len(temp_divs) > 1 else ""

        if not (label and label.replace("\xa0", "").strip()):
            raise ValueError("Label text is empty.")
    except:
        unlabeled_news.append(current_news_id)
        continue

    table = soup.find('table')
    if not table:
        labeled_news_no_accounts.append(current_news_id)
        continue

    tr_elements = table.find_all('tr')

    def clean_text(element):
        """Safely extract text content from an HTML element, replacing &nbsp; and stripping whitespace."""
        if element:
            text = element.get_text()
            if text and text.replace('\xa0', '').strip():
                return text.replace('\xa0', ' ').strip()
        return 'undefined'
                
    for r, tr in enumerate(tr_elements):
        try:
            th_elements = tr.find_all('th')
            td_elements = tr.find_all('td')

            account_info = {
                'row_table_number': r+1,
                'account_url': 'undefined',
                'account_name': 'undefined',
                'followers_number': 'undefined',
                'label': label,
                'news_id': current_news_id
            }

            if th_elements and len(th_elements) >= 3:
                if th_elements[1].select_one('a'):
                    account_info['account_url'] = th_elements[1].select_one('a')['href']
                    account_info['account_name'] = clean_text(th_elements[1])
                    account_info['followers_number'] = clean_text(th_elements[2])
                else:
                    continue
            
            elif td_elements and len(td_elements) >= 3:
                #already caputered in the last first extract proccess
                continue

            elif td_elements and len(td_elements) == 2:
                if td_elements[0].select_one('a'):
                    account_info['account_url'] = td_elements[0].select_one('a')['href']
                    account_info['account_name'] = clean_text(td_elements[0])
                    account_info['followers_number'] = clean_text(td_elements[1])
                else:
                    continue

            else:
                if current_news_id not in labeled_news_no_accounts:
                    labeled_news_no_accounts.append(current_news_id)
                continue

            accounts.append(account_info)

        except Exception as e:
            if current_news_id not in labeled_news_no_accounts:
                labeled_news_no_accounts.append(current_news_id)
            continue


In [101]:
print("number of extracted accounts: ",len(accounts))
print("number of unlabeled news:",len(unlabeled_news))
print("number of labeled news with no accounts",len(labeled_news_no_accounts))

number of extracted accounts:  40
number of unlabeled news: 0
number of labeled news with no accounts 0


In [102]:
accounts_df = pd.DataFrame(accounts)
accounts_df.head()

,row_table_number,account_url,account_name,followers_number,label,news_id
0,1,https://web.facebook.com/share/p/1BxwcLf8HN/,قوات العمل الخاص و هيئة العمليات 🇸🇩,(690) ألف متابع,مفبرك,22
1,2,https://web.facebook.com/share/p/19BBYkPd1t/,تجمع رحالين ترحال ليمون قو فرس,(569) ألف متابع,مفبرك,22
2,3,https://web.facebook.com/share/p/152CG2uU52o/,الجيش السوداني 🇸🇩,(361) ألف متابع,مفبرك,22
3,4,https://web.facebook.com/share/p/196YnjFB9d/,الانصرافي,(159) ألف متابع,مفبرك,22
4,1,https://web.facebook.com/groups/57262537958967...,الراكوبـــــه الســـــودانيه( ❤بيتنا الكبير يل...,(850) ألف متابع,مفبرك,63


In [103]:
main_df = pd.concat([main_df,accounts_df],ignore_index=True).sort_values(by=["news_id","row_table_number"],ascending=[True,True])

In [104]:
evaluate_df(main_df)

number of undefined labels:  0
number of undefined accounts name :  0
number of undefined followers_number :  1
number of undefined account_url :  0


#### merge extracted accounts with main_df

In [105]:
# save main_df
main_df.to_csv("accounts_data/accounts_info_file_filtered_extended_1.csv", index=False, encoding='utf-8-sig')

In [106]:
no_label_df = pd.read_csv("accounts_data/news_no_labels_file.csv")
no_label_df.columns = ["news_id"]
no_label_df.to_csv("accounts_data/news_no_labels_file.csv", index=False, encoding='utf-8-sig')

#### check if there is any news not included on any file yet

In [112]:
main_df = pd.read_csv("accounts_data/accounts_info_file_filtered_extended_1.csv")
label_no_table_df = pd.read_csv("accounts_data/labeled_news_with_no_tables.csv")
no_label_df = pd.read_csv("accounts_data/news_no_labels_file.csv")
urls_df =  pd.read_excel("news_id_url.xlsx")
#df4 = pd.read_csv("accounts_data/accounts_info_file_merged.csv")
#intersection

def unq_news_id(df):
    return list(df["news_id"].unique())
    
unq1 = unq_news_id(main_df)
unq2 = unq_news_id(label_no_table_df)
unq3 = unq_news_id(no_label_df)
total_news = unq_news_id(urls_df)

sum_news_number = len(unq1)+len(unq2)+len(unq3)
all_news_number = len(total_news)

print(f"sum of extracted news: {sum_news_number} , total number of news: {all_news_number} ")


sum of extracted news: 546 , total number of news: 547 


In [113]:
not_listed = set(total_news) - set(unq1+unq2+unq3)
temp_df = pd.DataFrame(not_listed)
label_no_table_df = pd.concat([label_no_table_df,temp_df], ignore_index=True)
label_no_table_df.to_csv("accounts_data/labeled_news_with_no_tables.csv", index=False, encoding='utf-8-sig')

We finished the process of extracting accounts from beam report website.